3/5/17 (played poker at angelas place; really thinking about egg freezing for bay now that we're 33 years old; lurm is back from korea and doesnt seem to be with james anymore; broder got a new job at google - it's nice to see him out of pantheon; lurm starts squarespace tomorrow; reviewing blockchains for fun)

3/11/17 (walked in central park with yule/andy in 20 degree weather; bay is sleeping next to me - we need to talk more in our marriage; hoping to close stae in the coming weeks; nyc board meetings killed vrb; lurm is in sxsw; bolbs quit pantheon and is heading to thailand for edisons wedding; tried out beef patties from my little pizzeria; spending time on blockchains bc trying to understand sovereign identity)

5/26/17 (went for a loop w bay on the promenade; smart cities dinner last night it was fun bc people were young and unpretentious; lurmie going to see jerry the bear this memorial day weekend - it's good she has friends; momo has been texting me nonstop with random things; in a bit of a rut hopefully not working this weekend will make things better; tiff/ed got engaged this tues - the engagement dinner was a bit awkward; jaemin the intern started this week - hes so likeable; reviewing sockets bc i know nothing about networking; reviewing pca versus autoencoders after coming across https://danluu.com/linear-hammer/; rented shaun of the dead; memorial day weekend and will be lazy with my bay the entire weekend; bought bose headphones for bolbs because he bought me $300 wallet - i miss my brother; sands st board interview tomorrow; working out of a/d/o in greenpoint, such a nice atmosphere with good lighting)

6/30/17 (finally pushed cylera over the finish line - found out gus is a fighter; emily will probably be reassigned; shed is out; bolbs will be here this weekend by himself - does this mean there is problem with laura; bay is exhausted and had to take a mental day; sands st apt closed; momo says she wants to retire)

## reference
* https://medium.com/@lhartikk/a-blockchain-in-200-lines-of-code-963cc1cc0e54#.f2isajv22
* blockchain for authentication http://www.talkcrypto.org/blog/portfolio/digital-signatures-and-the-blockchain/

## todo

* broadcast updates to blockchain?

In [3]:
from miner import Miner
from node import Node
from block import Block

# Creating the Blockchain

we initialize the blockchain ("ledger") with the genesis block. 

for our toy example, our ledger will be a python list containing Block objects.

In [4]:
genesis_block = Block(0, "none", 'laura pays $1 to steve', 'hello')
genesis_block.show()

>>> block index: 1
>>> previous hash: none
>>> current hash: hello
>>> transaction: laura pays $1 to steve


In [5]:
ledger = [genesis_block]

## step 1: creating blocks
nodes create a new Block for every new transaction/record.

In [6]:
Node_Alice = Node()
Node_Bob = Node()

In [7]:
block1 = Node_Alice.generate_new_block('steve pays $2 to michelle', ledger)
block1.show()

>>> block index: 2
>>> previous hash: hello
>>> current hash: -- placeholder hash --
>>> transaction: steve pays $2 to michelle


## step 2: attempt to add new block to the ledger

a new Block cannot be added to the blockchain until its hash demonstrates **proof of work**. 

in our toy example, an acceptable hash is one where h(previous hash + transaction) starts with the character "c". h(x) in this case is sha1 hash function.

for example, sha1("hello World11") is 'cc8e9d1fbe83d1ce4b1bf546ebff3b735e4ff500'. since this hash output starts with the character "c", it will be considered to be a valid **proof of work** and thus its associated Block can be added to the ledger.

in practice, an acceptable hash, usually sha256, must start with three zeros (eg "000abfds..."). 

In [8]:
# can new block be added to blockchain?
Node_Alice.update_blockchain(block1, ledger)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False
>>> update failed


False

## step 3: miners demonstrate proof of work
through brute force, miners find the correct hash that satisfies conditions. once the correct hash is identified, it will be assigned to that block, thereby enabling addition to the ledger.

In [9]:
Miner_Adam = Miner()

In [10]:
good_hash = Miner_Adam.calc_hash_seal(block1)
print "acceptable hash:", good_hash

acceptable hash: 111111111111111111111111111111111111111111111111111111111111111111111


In [11]:
# miner updates the block with the hash
Miner_Adam.update_hash(block1, good_hash)

>>> updating hash for block 2


In [13]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block1, ledger)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

# simulate another transaction

In [14]:
block2 = Node_Bob.generate_new_block('michelle pays $1 to joanna', ledger)
block2.show()

>>> block index: 3
>>> previous hash: 111111111111111111111111111111111111111111111111111111111111111111111
>>> current hash: -- placeholder hash --
>>> transaction: michelle pays $1 to joanna


In [15]:
Node_Bob.check_block(block2, ledger)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False


False

In [16]:
good_hash = Miner_Adam.calc_hash_seal(block2)
print "good_hash:", good_hash

good_hash: 11111111111


In [17]:
# since we've found the seal, we can update the block, thus approving it for blockchain addition
Miner_Adam.update_hash(block2, good_hash)

>>> updating hash for block 3


In [18]:
Node_Bob.check_block(block2, ledger)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True


True

In [19]:
# with the new hash, verify new block can be added to blockchain
Node_Bob.update_blockchain(block2, ledger)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

In [20]:
# examine history
for blocks in ledger:
    print blocks.fetch_transaction()

record 1 : laura pays $1 to steve
record 2 : steve pays $2 to michelle
record 3 : michelle pays $1 to joanna


In [21]:
# see if record 2 is valid
print "compare hash signatures", ledger[1].curr_hash == Miner.calc_hash_seal(ledger[1])

compare hash signatures True


# what is record 1 was altered?

In [86]:
# examine history
for b in bitcoin:
    print b.fetch_transaction()

record 1 : laura pays $1 to steve
record 2 : steve pays $2 to michelle
record 3 : michelle pays $1 to joanna


In [87]:
bitcoin[1].data = "dummy data"

In [88]:
# examine history
for b in bitcoin:
    print b.fetch_transaction()

record 1 : laura pays $1 to steve
record 2 : dummy data
record 3 : michelle pays $1 to joanna


In [89]:
# see if record 2 is valid
print "compare hash signatures", bitcoin[1].curr_hash == Miner.calc_hash_seal(bitcoin[1])

compare hash signatures False
